In [1]:
import hopsworks
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
import os
import numpy as np
import joblib
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

In [3]:
# # You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/196769
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Get features (data) from feature store
# bitcoin data
bitcoin_fg = fs.get_feature_group(name="bitcoin", version=1)
print(bitcoin_fg)
query = bitcoin_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="bitcoin",
                                  version=1,
                                  description="Read from bitcoin dataset",
                                  labels=["close"],
                                  query=query)
print(feature_view)

df_bitcoin = bitcoin_fg.read()
df_bitcoin = df_bitcoin.sort_values(by="date")
df_bitcoin.reset_index(drop=True,inplace=True)

print(df_bitcoin)
y_bitcoin = df_bitcoin['close'].values.reshape(-1,1)

Finished: Reading data from Hopsworks, using ArrowFlight (0.99s) 
                          date        close
0    2000-08-30 00:00:00+00:00   273.899994
1    2000-08-31 00:00:00+00:00   278.299988
2    2000-09-01 00:00:00+00:00   277.000000
3    2000-09-05 00:00:00+00:00   275.799988
4    2000-09-06 00:00:00+00:00   274.200012
...                        ...          ...
5854 2023-12-29 00:00:00+00:00  2062.399902
5855 2024-01-02 00:00:00+00:00  2064.399902
5856 2024-01-03 00:00:00+00:00  2034.199951
5857 2024-01-04 00:00:00+00:00  2042.300049
5858 2024-01-05 00:00:00+00:00  2042.400024

[5859 rows x 2 columns]


In [5]:
# forecast setting
n_forecast = 90  # length of output sequences (forecast period)
n_lookback = 3*n_forecast  # length of input sequences (lookback period, should be 3 times longer than forecast period)

In [6]:
# Model Training for bitcoin
scaler_bitcoin = MinMaxScaler(feature_range=(0, 1))
scaler_bitcoin = scaler_bitcoin.fit(y_bitcoin)
y_bitcoin = scaler_bitcoin.transform(y_bitcoin)

X_bitcoin = []
Y_bitcoin = []

for i in range(n_lookback, len(y_bitcoin) - n_forecast + 1):
    X_bitcoin.append(y_bitcoin[i - n_lookback: i])
    Y_bitcoin.append(y_bitcoin[i: i + n_forecast])

X_bitcoin = np.array(X_bitcoin)
Y_bitcoin = np.array(Y_bitcoin)
print(X_bitcoin.shape, Y_bitcoin.shape)

# fit / train the model
model_bitcoin = Sequential()
model_bitcoin.add(LSTM(units=128, return_sequences=True, input_shape=(n_lookback, 1)))
model_bitcoin.add(LSTM(units=64, return_sequences=True))
model_bitcoin.add(LSTM(units=64, return_sequences=False))
model_bitcoin.add(Dense(n_forecast))
model_bitcoin.compile(loss='mean_squared_error', optimizer='adam')
model_bitcoin.fit(X_bitcoin, Y_bitcoin, epochs=20, batch_size=32, verbose=1)

(5500, 270, 1) (5500, 90, 1)
Epoch 1/20
172/172 [==============================] - 44s 236ms/step - loss: 0.0211
Epoch 2/20
172/172 [==============================] - 41s 237ms/step - loss: 0.0024
Epoch 3/20
172/172 [==============================] - 42s 242ms/step - loss: 0.0022
Epoch 4/20
172/172 [==============================] - 41s 240ms/step - loss: 0.0022
Epoch 5/20
172/172 [==============================] - 42s 243ms/step - loss: 0.0023
Epoch 6/20
172/172 [==============================] - 42s 243ms/step - loss: 0.0020
Epoch 7/20
172/172 [==============================] - 41s 241ms/step - loss: 0.0021
Epoch 8/20
172/172 [==============================] - 42s 243ms/step - loss: 0.0020
Epoch 9/20
172/172 [==============================] - 42s 242ms/step - loss: 0.0021
Epoch 10/20
172/172 [==============================] - 41s 240ms/step - loss: 0.0020
Epoch 11/20
172/172 [==============================] - 42s 242ms/step - loss: 0.0020
Epoch 12/20
172/172 [========================

In [27]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()
model_dir="bitcoin_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model_bitcoin, model_dir + "/bitcoin_model.pkl")
# fig.savefig(model_dir + "/wine_confusion_matrix.png")

# # Create an entry in the model registry that includes the model's name, desc, metrics
bitcoin_model = mr.python.create_model(
    name="bitcoin_model",
    # metrics={"accuracy" : metrics['accuracy']},
    # model_schema=model_schema,
    description="bitcoin Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
bitcoin_model.save(model_dir)


Connected. Call `.close()` to terminate connection gracefully.
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-01-11 21:02:06         3468
metadata.

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/196769/models/gold_model/3


Model(name: 'gold_model', version: 3)